# [MERGED] With 2024-08-30-classifier.ipynb

Keeping this for archival reasons


In [1]:
import pandas as pd
from functools import reduce
from pathlib import Path

In [2]:
#! beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM
#!beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM

In [3]:
LEXICAL_FEATS_PATH = Path("dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features")
DATASET_PATH = Path(
    "dataset/01J6KBM2VCM9EQ7MER26VBXCCM/helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl"
)

Let's just merge the lexical features


In [4]:
lexical_features = [
    "rouge",
    "bertscore",
    "bertscore_length",
    "entity_sim",
    "cosine_sim",
    "prompt_len",
    "len_longer",
    "len_shorter",
    "token_len_diff",
]
lexical_feature_files = [
    file
    for file in LEXICAL_FEATS_PATH.glob("*.jsonl")
    if any(file.stem in feat for feat in lexical_features)
]
print(lexical_feature_files)

[PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/rouge.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/bertscore.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/entity_sim.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/len_shorter.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/bertscore_length.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/prompt_len.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/len_longer.jsonl'), PosixPath('dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features/cosine_sim.jsonl')]


In [5]:
lexical_feats_df = reduce(
    lambda left, right: left.merge(
        right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
    ),
    [pd.read_json(file, lines=True) for file in lexical_feature_files],
)

We don't save the other Bowen's Features, so might need to add that manually


In [6]:
df = pd.read_json(DATASET_PATH, lines=True).rename(columns={"prompt_hash": "id"})

In [7]:
finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
    columns=["prompt", "completion_a", "completion_b"]
)

In [8]:
finaldf.to_csv("helpsteer2_featureset.csv", index=False)

In [9]:
finaldf.head(10)

,id,text,response_a,response_b,pref_human,pref_gpt4,rating_human,rating_gpt4,completions,subject_of_expertise,...,type_of_in_context_material,format_constraints,rouge,bertscore,entity_sim,len_shorter,bertscore_length,prompt_len,len_longer,cosine_sim
0,000be890ad02c46a3554c77f098f151f,In what ways can researching something be help...,Engaging in research is a valuable endeavor th...,There are several reasons why doing research c...,B-is-clearly-better,A-is-clearly-better,"[2.83, 6.28]","[6.09, 5.98]",[Engaging in research is a valuable endeavor t...,"[Education, Psychology]",...,[],"[#words <= 200, style = formal]",0.328704,0.799727,1.000000,210,0.435609,25,376,0.709485
1,001458ba4f2925f2bc6ae9d8d197d543,Write an inspiring song about alena wicker for...,"Here's a song inspired by Alena Wicker, a youn...",Here's an inspiring song about Alena Wicker fo...,B-is-clearly-better,B-is-clearly-better,"[3.73, 4.77]","[1.5899999999999999, 2.25]","[Here's a song inspired by Alena Wicker, a you...",[Performing arts],...,[],"[content = sone, style = inspiring, include: t...",0.463866,0.842092,0.333333,442,0.724686,44,503,0.844983
2,0014db4cd30cad106320d9043f45d607,Hi! I am a Hypnotist and transformational / mi...,They want a solution to their problems. They w...,They want a solution to lead a more fulfilling...,B-is-clearly-better,A-is-clearly-better,"[4.77, 5.95]","[5.87, 5.62]",[They want a solution to their problems. They ...,"[Business, Psychology]",...,[],[],0.150259,0.772617,0.000000,56,0.000456,209,480,0.508719
3,003af4a043a0cb2186bc448a290d35f6,Australian Defence Force Military Police membe...,"Yes, members of the Australian Defence Force M...","Yes, members of the Australian Defence Force M...",Tie,B-is-clearly-better,"[6.28, 6.28]","[5.62, 6.09]","[Yes, members of the Australian Defence Force ...","[Military sciences, Law]",...,[],[],0.510121,0.881362,0.500000,96,0.268603,30,217,0.872330
4,003dd8aa40b027ae755e47917c89c0a3,Heather: I want to attend the music festival t...,It is unclear if all girls have enough money t...,Yes\n\nBoth Heather and Kara state that they h...,A-is-clearly-better,A-is-clearly-better,"[6.14, 3.02]","[5.13, 0.64]",[It is unclear if all girls have enough money ...,"[Performing arts, Economics]",...,[],[],0.474227,0.866347,0.600000,52,0.671132,107,68,0.839198
5,0042983fc394379d0a650706563946c3,I am working on a website that uses a single-p...,There are several ways to pass data from one v...,There are several ways to pass data from one v...,A-is-clearly-better,A-is-clearly-better,"[5.95, 5.24]","[6.09, 5.24]",[There are several ways to pass data from one ...,[Computer sciences],...,[],[],0.355748,0.818510,0.500000,297,0.129798,59,827,0.775792
6,004303a1ee663ebb9f48bb3fba63b5ca,explain the steps of e waste management of com...,The reactions for supercapacitors involve the ...,Supercapacitors are electrochemical capacitors...,Tie,Tie,"[6.75, 6.75]","[6.75, 6.75]",[The reactions for supercapacitors involve the...,"[Chemical engineering, Physics]",...,[],[],0.640227,0.856557,1.000000,243,0.765108,436,297,0.926079
7,004506a4038d7b9c429b0a67553698e5,"Using scholarly language, produce a detailed c...","The study of word meanings, also known as sema...","The study of word meanings, also known as lexi...",A-is-clearly-better,B-is-clearly-better,"[6.61, 2.37]","[5.79, 6.09]","[The study of word meanings, also known as sem...","[Linguistics and language, Literature, History]",...,[],[content = chronology],0.364508,0.785917,0.035714,634,0.362111,44,1131,0.712766
8,004d56a35cf1a852955350b87ac901c5,You are a pregnancy health & nutrition expert ...,"I. Introduction\n \n\n Acid reflux, also known...","I. Introduction\n \n\n Acid reflux, also known...",Tie,B-is-clearly-better,"[2.45, 2.45]","[4.2, 4.5]","[I. Introduction\n \n\n Acid reflux, also know...",[Medicine and health],...,[],[],0.636872,0.883891,0.000000,785,0.608232,947,1254,0.957600
9,004fa50d78e24f7022a3f02b4727a730,Hello ChatGPT. You are about to immerse yourse...